# Decision Tree Object Classifier
## Loading Libraries

In [1]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_wine
np.set_printoptions(precision=4)
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## Dataset

In [41]:
# SQUARE
with open('../Control/data_pos_trial.json') as f:
    full_data_dict_square = json.load(f)
# HEXAGON
with open('../Control/data_pos_trial_hexagon.json') as f:
    full_data_dict_hexagon = json.load(f)
# CIRCLE
with open('../Control/data_pos_trial_circle.json') as f:
    full_data_dict_circle = json.load(f)
# RECTANGLE
with open('../Control/data_pos_trial_rectangle.json') as f:
    full_data_dict_rectangle = json.load(f)

number_trials = len(full_data_dict_square) + len(full_data_dict_rectangle) 
number_trials += len(full_data_dict_circle) + len(full_data_dict_hexagon) 
data_array  = np.empty((0,5), int) # 5 --> LF1_2;TIME_12;CLASS

poses = [60,65,70]
gaps = [34,42,50]
dims = [20,25,30]
trials = [1,2]
data_list=[]

# class --> 0:square 1:hexagon 2:circle 3:rectangle
# square and hexagon same trials
for trial in trials:
    for gap in gaps:
        for dim in dims:
            for pose in poses:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+"_"+str(trial)
                data_dict_square = full_data_dict_square[dict_key]
                data_trial = np.array([[data_dict_square['LF_motion1'][0],
                                        data_dict_square['LF_motion1'][1],
                                        data_dict_square['LF_motion2'][0],
                                        data_dict_square['LF_motion2'][1],
                                        0]])
                data_array = np.append(data_array, data_trial, axis=0)
                data_dict_hexagon = full_data_dict_hexagon[dict_key]
                data_trial = np.array([[data_dict_hexagon['LF_motion1'][0],
                                        data_dict_hexagon['LF_motion1'][1],
                                        data_dict_hexagon['LF_motion2'][0],
                                        data_dict_hexagon['LF_motion2'][1],
                                        1]])
                data_array = np.append(data_array, data_trial, axis=0)
# for circle the dim 20 gaps 50 only have the 70 pose
dims_circle = [20,25]
for trial in trials:
    for gap in gaps:
        for dim in dims_circle:
            for pose in poses:
                if not ((dim == 20 and gap == 50 and pose == 60) or 
                    (dim == 20 and gap == 50 and pose == 65)):
                    dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+"_"+str(trial)
                    data_dict_circle = full_data_dict_circle[dict_key]
                    data_trial = np.array([[data_dict_circle['LF_motion1'][0],
                                            data_dict_circle['LF_motion1'][1],
                                            data_dict_circle['LF_motion2'][0],
                                            data_dict_circle['LF_motion2'][1],
                                            2]])
                    data_array = np.append(data_array, data_trial, axis=0)
# for rectangle there is only one object of dim 20x25
dims_rectangle = [[20,25],[25,20]]
for trial in trials:
    for gap in gaps:
        for dim in dims_rectangle:
            for pose in poses:
                dict_key = "dim_"+str(dim[0])+"_"+str(dim[1])+"_pose_"+str(pose)+"_gap_"+str(gap)+"_"+str(trial)
                data_dict_rectangle = full_data_dict_rectangle[dict_key] 
                data_trial = np.array([[data_dict_rectangle['LF_motion1'][0],
                                        data_dict_rectangle['LF_motion1'][1],
                                        data_dict_rectangle['LF_motion2'][0],
                                        data_dict_rectangle['LF_motion2'][1],
                                        3]])
                data_array = np.append(data_array, data_trial, axis=0)

[[list([2329, 2328, 2323, 2316, 2309, 2299, 2291, 2283, 2276, 2268, 2259, 2251, 2243, 2235, 2227, 2219, 2211, 2203, 2195, 2186, 2178, 2170, 2162, 2154, 2146, 2138, 2130, 2121, 2113, 2106, 2099, 2092, 2084, 2076, 2069, 2061, 2053, 2045, 2037, 2029, 2021, 2013, 2005, 1997, 1988, 1980, 1972, 1965, 1957, 1949, 1940, 1932, 1924, 1916, 1908, 1900, 1892, 1884, 1876, 1868, 1860, 1852, 1844, 1836, 1828, 1820, 1812, 1804, 1796, 1788, 1780])
  list([6.044645071029663, 6.09264612197876, 6.140779972076416, 6.1886560916900635, 6.236785173416138, 6.28463888168335, 6.332605838775635, 6.380626916885376, 6.428593873977661, 6.476608991622925, 6.5246148109436035, 6.572950124740601, 6.620550870895386, 6.668578147888184, 6.717057943344116, 6.764586925506592, 6.812600135803223, 6.860589027404785, 6.908630132675171, 6.956571817398071, 7.004590034484863, 7.052560091018677, 7.1006011962890625, 7.1486289501190186, 7.196620941162109, 7.2445759773254395, 7.292625904083252, 7.340538024902344, 7.388638973236084, 7.4

<ipython-input-41-6f02b21dd25d>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_trial = np.array([[data_dict_square['LF_motion1'][0],
<ipython-input-41-6f02b21dd25d>:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_trial = np.array([[data_dict_hexagon['LF_motion1'][0],
<ipython-input-41-6f02b21dd25d>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object

# Random Forest Classifier
https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76

In [36]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(train, train_labels)